# Plot maps of SWOT data, algorithm runs, and mean flow post integrator

by Mike Durand. Confluence summit at U Mass, April 2024

Edited by Elisa (Ellie) Friedmann, Jonathan Flores, Craig Brinkerhoff April 2024

## Set up Libraries and Directories

In [1]:
import os,sys
import netCDF4
from pathlib import Path
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
from netCDF4 import chartostring


In [2]:
# set up SWOT DAWG viz
sys.path.append('/nas/cee-water/cjgleason/stevec/SWOTdawgDISTRO/notebooks/Umass')
from swotdawgviz.swotdawgviz import io as sdvio
from swotdawgviz.swotdawgviz import maps as sdvm

This library is supposed to run on a compatible Jupyter notebook. Please be aware that some method may not work


In [3]:
os.getcwd()

'/nas/cee-water/cjgleason/ellie/SWOT'

In [4]:
# set up directories
DataDir=Path('.')

## Open files

In [5]:
results = netCDF4.Dataset("/nas/cee-water/cjgleason/SWOT_Q_UMASS/na_sword_v16_SOS_results_EOD_day1.nc", format="NETCDF4")

#print("Results Group")
#print(results, "\n")


## Parse Ohio basin in SoS

The Ohio is L4=7426

In [ ]:
reaches = results.groups['reaches']
print("Reaches Group")
print(reaches, "\n")

reachids=reaches['reach_id'][:]

reachid_strs=[]

reachids_oh=[]
reachids_int_oh=[]

for reachid in reachids:
    if str(reachid)[0:4]=='7426':
        reachids_oh.append(str(reachid))
        reachids_int_oh.append(int(reachid))

In [ ]:
reachids

## Explore integrator data stored in SoS

In [ ]:
moi = results.groups['moi']
print("MOI Group")
print(moi, "\n")

In [ ]:
print("HiVDI in MOI")
print(moi['hivdi'], "\n")

In [ ]:
print("Basin scale discharge from HiVDI in MOI")
print(moi['hivdi']['qbar_basinScale'], "\n")

In [ ]:
moi['hivdi']['qbar_basinScale'][14954].data

## Initialize SWOT DAWG Viz map

In [ ]:
import geopandas as gpd

rch = gpd.read_file("/nas/cee-water/cjgleason/miked/umass_workshop/sword_shp/na_sword_reaches_hb74_v16.shp")
rch.plot()
print(len(rch))

In [ ]:
rch[rch.reach_id.isin(reachids.data)].plot()

In [ ]:
# Open the priors file:
priors = netCDF4.Dataset("/nas/cee-water/cjgleason/SWOT_Q_UMASS/na_sword_v16_SOS_priors.nc", format="NETCDF4")

gauge_reach = priors["USGS"]["USGS_reach_id"][:]
print("Gauge reach identifiers:")
print(gauge_reach)

reach_overlap = np.intersect1d(gauge_reach, reachids.data)
print("Overlapping reaches:")
print(reach_overlap)
reach_overlap

In [ ]:
len(reach_overlap)

In [ ]:
gauged_reaches = rch[rch.reach_id.isin(reach_overlap)]
print(len(gauged_reaches))
gauged_reaches.plot()

In [ ]:
gauged_reaches

results#["hivdi"]["Q"]

In [ ]:
# create swotdawgviz map with just the reaches in RL mapped
sword_hb_reaches = sdvio.SwordShapefile("/nas/cee-water/cjgleason/miked/umass_workshop/sword_shp/na_sword_reaches_hb74_v16.shp",reachids_int_oh)
rmap = sdvm.ReachesMap(sword_hb_reaches.dataset)
ridmap = rmap.get_centerlines_map()
ridmap

## Map mean flow for one algorithm

In [ ]:
#hidvi
Qbar_hi=dict()
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    data = np.ma.getdata(results["hivdi"]["Q"][idx])[0]
    if data.max() > 0:
        Qbar_hi[reachid] = 1
    else:
        Qbar_hi[reachid] = 0
        
#momma
Qbar_momma=dict()
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    data = np.ma.getdata(results["momma"]["Q"][idx])[0]
    if data.max() > 0:
        Qbar_momma[reachid] = 1
    else:
        Qbar_momma[reachid] = 0
        
#sad
Qbar_sad=dict()
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    data = np.ma.getdata(results["sad"]["Qa"][idx])[0]
    if data.max() > 0:
        Qbar_sad[reachid] = 1
    else:
        Qbar_sad[reachid] = 0
        
        
#sic4dvar
Qbar_sic=dict()
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    data = np.ma.getdata(results["sic4dvar"]["Q_da"][idx])[0]
    #print(data)
    if data.max() > 0:
        Qbar_sic[reachid] = 1
    else:
        Qbar_sic[reachid] = 0

In [ ]:
# hidvi_ids = [int(x) for x in list(Qbar_hi.keys())]
# print(len(rch[rch.reach_id.isin(hidvi_ids)]))
# rch[rch.reach_id.isin(hidvi_ids)].plot()

In [ ]:
# add integrated MetroMan discharge to the rmap object
rmap._dataset['HiVDI']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbar_hi[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['HiVDI']]=Qbar_hi[str(reachid)]
    

rmap._json_dataset = rmap._dataset.to_json()    
hi_map = rmap.get_centerlines_map(varname="HiVDI",varlimits=[0,1],cmap=['r','b'])
hi_map

In [ ]:
# add integrated MetroMan discharge to the rmap object
rmap._dataset['Momma']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbar_momma[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['Momma']]=Qbar_momma[str(reachid)]
rmap._json_dataset = rmap._dataset.to_json()
momma_map = rmap.get_centerlines_map(varname="Momma",varlimits=[0,1],cmap=['r','b'])

momma_map

In [ ]:
momma_df = pd.Series(Qbar_momma)
print(len(momma_df[momma_df>0]))

In [ ]:
# add integrated MetroMan discharge to the rmap object
rmap._dataset['Sad']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbar_sad[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['Sad']]=Qbar_sad[str(reachid)]
rmap._json_dataset = rmap._dataset.to_json()
sad_map = rmap.get_centerlines_map(varname="Sad",varlimits=[0,1],cmap=["r", "b"])
sad_df = pd.Series(Qbar_sad)
print(len(sad_df[sad_df>0]))
sad_map

In [ ]:
# add integrated MetroMan discharge to the rmap object
rmap._dataset['Sic']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbar_sic[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['Sic']]=Qbar_sic[str(reachid)]
rmap._json_dataset = rmap._dataset.to_json()
sic_map = rmap.get_centerlines_map(varname="Sic",varlimits=[0,1],cmap=["r", "b"])
sic_df = pd.Series(Qbar_sic)
print(len(sic_df[sic_df>0]))
sic_map

In [ ]:
#all
Qbars=dict()
algo = {}
for reachid in reachids_oh:    
    idx = np.where(results['reaches']['reach_id'][:] == np.int64(reachid) )
    counter = []
    temp_algo = ""
    if np.ma.getdata(results["hivdi"]["Q"][idx])[0].max() > 0:
        counter.append(1)
        temp_algo = f"hivdi"
    if np.ma.getdata(results["momma"]["Q"][idx])[0].max() > 0:
        counter.append(1)
        temp_algo = f"{temp_algo}, momma"
    if np.ma.getdata(results["sad"]["Qa"][idx])[0].max() > 0:
        counter.append(1)
        temp_algo = f"{temp_algo}, sad"
    if np.ma.getdata(results["sic4dvar"]["Q_da"][idx])[0].max() > 0:
        counter.append(1)
        temp_algo = f"{temp_algo}, sic4dvar"
        
    Qbars[reachid] = sum(counter)
    algo[reachid] = temp_algo

#SHOW
df = pd.Series(Qbars)
df = df[df>1]
df.max()
#Qbars 

In [ ]:
from matplotlib import cm
import numpy as np
import random

class ColormapStyleFunction:
    """Object to handle colormap style functions
    """
    
    def __init__(self, cmap, attribute,randomcolors=False):
        self._cmap = cmap
        self._attribute = attribute
        self._randomcolors = randomcolors
        
    def __call__(self, x):
        if self._randomcolors:
            #hexcolor = '#ff0000'
            hexcolor="#"+''.join([random.choice('0123456789ABCDEF') for i in range(6) ] )
        else:
            hexcolor = self._cmap(x["properties"][self._attribute])

        return {'color': hexcolor, 'weight' : 3}
    
def get_centerlines_map(self, varname=None, cmap=None, tooltip_attributes=None, add_to_map=None, varlimits=[None, None]):
        """Build a map width reaches as centerlines colored with values of a variable
        
        Parameters
        ----------
        varname : str
            Name of the variable used for coloring
        width_attribute : str
            Name of the variable for the width
        cmap : branca.Colormap
            Colormap used for coloring
        tooltip_attributes : list or None
            List of variables to display using ToolTip
        """
        
        # Set default values for unset parameters
        if cmap is None and varname is not None:
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            #cmap = branca.colormap.linear.YlOrRd_09.scale(self._dataset[varname].min(),
            #                                              self._dataset[varname].max())
            cmap = branca.colormap.linear.YlOrRd_09.scale(varlimits[0],
                                                          varlimits[1])
            #cmap = branca.colormap.ColorMap(vmin=0.0, vmax=1.0, caption='', max_labels=10)
            #cmap = branca.colormap.LinearColormap(['blue', 'yellow', 'red']).scale(varlimits[0], varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])
            
        elif isinstance(cmap, list):
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            cmap = branca.colormap.LinearColormap(cmap).scale(varlimits[0],
                                                              varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])

        if tooltip_attributes is None:
            if varname is None:
                tooltip_attributes = ["reach_id"]
            else:
                tooltip_attributes = ["reach_id", varname]

        if add_to_map is None:
        
            # Retrieve bounding box and center
            bounds = self._dataset.geometry.total_bounds.tolist()
            center = (0.5 * (bounds[1] + bounds[3]), 0.5 * (bounds[0] + bounds[2]))
            
            # Create map
            new_map = folium.Map(location=center, tiles=self._tiles, zoom_start=6)
            parent_map = new_map
            
        else:
            
            parent_map = add_to_map
                       
        # Add layer
        tooltip = folium.GeoJsonTooltip(fields=tooltip_attributes)

        if varname is None:
            style_function = ColormapStyleFunction(cmap, varname, randomcolors=True)
        else:
            style_function = ColormapStyleFunction(cmap, varname)

        folium.GeoJson(self._json_dataset,
                       style_function=style_function,
                       tooltip=tooltip,
                       name="Test").add_to(parent_map)

        if varname is not None:
            
            # Add colorbar
            colormap = cmap.to_step(n=4)
            colormap.caption = varname
            colormap.add_to(parent_map)

        #if add_to_map is None:
            #parent_map.fit_bounds(self._dataset.total_bounds.tolist())
        
        if add_to_map is None:
            return new_map
        
rmap._dataset['all']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbars[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['all']]=Qbars[str(reachid)]
    #if not algo[str(reachid)]:
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['algo']]=algo[str(reachid)]
rmap._dataset['wse'] = np.round(rmap._dataset['wse'],2)
rmap._json_dataset = rmap._dataset.to_json()
all_map = rmap.get_centerlines_map(varname="all",varlimits=[0,4],cmap=["red","orange", "yellow", "green", "blue"]
                                   ,tooltip_attributes=['reach_id','all','algo','wse','width','river_name']) 
all_map

In [ ]:
rmap._dataset

# Adding attributes

# Gauge ID

In [ ]:
#Import validation gauge dataset and show
priors = netCDF4.Dataset("/nas/cee-water/cjgleason/SWOT_Q_UMASS/na_sword_v16_SOS_priors.nc", format="NETCDF4")
gauge_reach = priors["USGS"]["USGS_reach_id"][:]
print(gauge_reach)
print(list(chartostring(priors['USGS']['USGS_id'][:]))[0:10])

In [ ]:
# initializing lists
test_keys = list((gauge_reach))
test_keys = [str(element) for element in gauge_reach]
test_values = list(chartostring(priors['USGS']['USGS_id'][:]))
print(type(test_values))

# All gauges to reaches dict
usgs_dict = {test_keys[i]: test_values[i] for i in range(len(test_keys))}
print(len(usgs_dict))
#print(usgs_dict)

#Make blank Ohio Dictionary to fit with the map geodataframe
#fill with values of the same datatype as the eventual values
value = ''
size = len(reachids_oh)
nans = [value] * size
#print(nans)

ohio_dict = {reachids_oh[i]: nans[i] for i in range(len(reachids_oh))}
#print(ohio_dict)


merged = {**ohio_dict, **usgs_dict}
print(len(merged))


ohio_gauges = {key: merged[key] for key in reachids_oh}
print(len(reachids_oh))
print(len(ohio_gauges))



# WSE

In [ ]:
def process_nc_files(folder_path, var1_name, var2_name):
    data_dict = {}

    # Loop over each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".nc"):
            file_path = os.path.join(folder_path, filename)
            
            # Open the NetCDF file
            with netCDF4.Dataset(file_path, 'r') as nc_file:
                # Extract the variables
                var1 = str(nc_file[var1_name][0].min())
                var2 = nc_file[var2_name][:].mean()
                if np.ma.is_masked(var2):
                    var2 = float(-999999999999)
                # Store the variables in the dictionary
                data_dict[var1] = var2
    
    return data_dict

# Example usage
folder_path = '/nas/cee-water/cjgleason/SWOT_Q_UMASS/mnt/input/swot/'
var1_name = 'reach/reach_id'
var2_name = 'reach/wse'
wse_swot = process_nc_files(folder_path, var1_name, var2_name)

#Make blank Ohio Dictionary
value = -999999999999
size = len(reachids_oh)
nans = [value] * size
#print(nans)

ohio_dict = {reachids_oh[i]: nans[i] for i in range(len(reachids_oh))}
#print(ohio_dict)

print(len(wse_swot))
merged_wse_swot = {**ohio_dict, **wse_swot}
print(len(merged_wse_swot))


wse_swot_oh = {key: merged_wse_swot[key] for key in reachids_oh}
print(len(reachids_oh))
print(len(wse_swot_oh))

for k, v in wse_swot_oh.items():
    #if wse_swot_oh[k] != np.nan:
    wse_swot_oh[k] = float(v)

#print(wse_swot_oh)

# Dark_frac

In [ ]:
def process_nc_files(folder_path, var1_name, var2_name):
    data_dict = {}

    # Loop over each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".nc"):
            file_path = os.path.join(folder_path, filename)
            
            # Open the NetCDF file
            with netCDF4.Dataset(file_path, 'r') as nc_file:
                # Extract the variables
                var1 = str(nc_file[var1_name][0].min())
                var2 = nc_file[var2_name][:].mean()
                if np.ma.is_masked(var2):
                    var2 = -999999999999
                # Store the variables in the dictionary
                data_dict[var1] = var2
    
    return data_dict

# Example usage
folder_path = '/nas/cee-water/cjgleason/SWOT_Q_UMASS/mnt/input/swot/'
var1_name = 'reach/reach_id'
var2_name = 'reach/dark_frac'
dark_frac_swot = process_nc_files(folder_path, var1_name, var2_name)

#Make blank Ohio Dictionary
value = -999999999999
size = len(reachids_oh)
nans = [value] * size
#print(nans)

ohio_dict = {reachids_oh[i]: nans[i] for i in range(len(reachids_oh))}
#print(ohio_dict)

print(len(dark_frac_swot))
merged_dark_frac_swot = {**ohio_dict, **dark_frac_swot}
print(len(merged_dark_frac_swot))


dark_frac_swot_oh = {key: merged_dark_frac_swot[key] for key in reachids_oh}
print(len(reachids_oh))
print(len(dark_frac_swot_oh))

for k, v in dark_frac_swot_oh.items():
    #if dark_frac_swot_oh[k] != np.nan:
    dark_frac_swot_oh[k] = float(v)

#print(dark_frac_swot_oh)

# Width

In [ ]:
def process_nc_files(folder_path, var1_name, var2_name):
    data_dict = {}

    # Loop over each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".nc"):
            file_path = os.path.join(folder_path, filename)
            
            # Open the NetCDF file
            with netCDF4.Dataset(file_path, 'r') as nc_file:
                # Extract the variables
                var1 = str(nc_file[var1_name][0].min())
                var2 = nc_file[var2_name][:].mean()
                if np.ma.is_masked(var2):
                    var2 = -999999999999
                # Store the variables in the dictionary
                data_dict[var1] = var2
    
    return data_dict

# Example usage
folder_path = '/nas/cee-water/cjgleason/SWOT_Q_UMASS/mnt/input/swot/'
var1_name = 'reach/reach_id'
var2_name = 'reach/width'
width_swot = process_nc_files(folder_path, var1_name, var2_name)


#Make blank Ohio Dictionary
value = -999999999999
size = len(reachids_oh)
nans = [value] * size
#print(nans)

ohio_dict = {reachids_oh[i]: nans[i] for i in range(len(reachids_oh))}
#print(ohio_dict)

print(len(width_swot))
merged_width_swot = {**ohio_dict, **width_swot}
print(len(merged_width_swot))


width_swot_oh = {key: merged_width_swot[key] for key in reachids_oh}
print(len(reachids_oh))
print(len(width_swot_oh))

for k, v in width_swot_oh.items():
    #if width_swot_oh[k] != np.nan:
    width_swot_oh[k] = float(v)

#print(width_swot_oh)

# SLOPE SWOT

In [ ]:
def process_nc_files(folder_path, var1_name, var2_name):
    data_dict = {}

    # Loop over each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".nc"):
            file_path = os.path.join(folder_path, filename)
            
            # Open the NetCDF file
            with netCDF4.Dataset(file_path, 'r') as nc_file:
                # Extract the variables
                var1 = str(nc_file[var1_name][0].min())
                var2 = (nc_file[var2_name][:].mean())
                if np.ma.is_masked(var2):
                    var2 = -999999999999
                # Store the variables in the dictionary
                data_dict[var1] = var2
    
    return data_dict

# Example usage
folder_path = '/nas/cee-water/cjgleason/SWOT_Q_UMASS/mnt/input/swot/'
var1_name = 'reach/reach_id'
var2_name = 'reach/slope2'
slope_swot = process_nc_files(folder_path, var1_name, var2_name)

#Make blank Ohio Dictionary
value = -999999999999
size = len(reachids_oh)
nans = [value] * size
#print(nans)

ohio_dict = {reachids_oh[i]: nans[i] for i in range(len(reachids_oh))}
#print(ohio_dict)

print(len(slope_swot))
merged_slope_swot = {**ohio_dict, **slope_swot}
print(len(merged_slope_swot))


slope_swot_oh = {key: merged_slope_swot[key] for key in reachids_oh}
print(len(reachids_oh))
print(len(slope_swot_oh))


for k, v in slope_swot_oh.items():
    #if slope_swot_oh[k] == np.nan:
    slope_swot_oh[k] = float(v)

#print(slope_swot_oh)

# SLOPE SWORD

In [ ]:
import geopandas as gpd

sword = gpd.read_file("/nas/cee-water/cjgleason/miked/umass_workshop/sword_shp/na_sword_reaches_hb74_v16.shp")
print(sword.columns.values.tolist())
#print(sword.reach_id)

#Do some data type switching
reachds_oh_int = [int(element) for element in reachids_oh]
sword_oh = sword[sword['reach_id'].isin(reachds_oh_int)]
sword_oh['reach_id'] = sword_oh['reach_id'].astype(str)

#Make dictionary of sword reaches and slopes
slope_all = pd.Series(sword_oh.slope.values,index=(sword_oh.reach_id)).to_dict()
#print(len(slope_all))


merged_slope = {**ohio_dict, **slope_all}
print(len(merged_slope))


slopes = {key: merged_slope[key] for key in reachids_oh}
print(len(reachids_oh))


# MAPS

In [ ]:
from matplotlib import cm
import numpy as np
import random

class ColormapStyleFunction:
    """Object to handle colormap style functions
    """
    
    def __init__(self, cmap, attribute,randomcolors=False):
        self._cmap = cmap
        self._attribute = attribute
        self._randomcolors = randomcolors
        
    def __call__(self, x):
        if self._randomcolors:
            #hexcolor = '#ff0000'
            hexcolor="#"+''.join([random.choice('0123456789ABCDEF') for i in range(6) ] )
        else:
            hexcolor = self._cmap(x["properties"][self._attribute])

        return {'color': hexcolor, 'weight' : 3}
    
def get_centerlines_map(self, varname=None, cmap=None, tooltip_attributes=None, add_to_map=None, varlimits=[None, None]):
        """Build a map width reaches as centerlines colored with values of a variable
        
        Parameters
        ----------
        varname : str
            Name of the variable used for coloring
        width_attribute : str
            Name of the variable for the width
        cmap : branca.Colormap
            Colormap used for coloring
        tooltip_attributes : list or None
            List of variables to display using ToolTip
        """
        
        # Set default values for unset parameters
        if cmap is None and varname is not None:
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            #cmap = branca.colormap.linear.YlOrRd_09.scale(self._dataset[varname].min(),
            #                                              self._dataset[varname].max())
            cmap = branca.colormap.linear.YlOrRd_09.scale(varlimits[0],
                                                          varlimits[1])
            #cmap = branca.colormap.ColorMap(vmin=0.0, vmax=1.0, caption='', max_labels=10)
            #cmap = branca.colormap.LinearColormap(['blue', 'yellow', 'red']).scale(varlimits[0], varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])
            
        elif isinstance(cmap, list):
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            cmap = branca.colormap.LinearColormap(cmap).scale(varlimits[0],
                                                              varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])

        if tooltip_attributes is None:
            if varname is None:
                tooltip_attributes = ["reach_id"]
            else:
                tooltip_attributes = ["reach_id", varname]

        if add_to_map is None:
        
            # Retrieve bounding box and center
            bounds = self._dataset.geometry.total_bounds.tolist()
            center = (0.5 * (bounds[1] + bounds[3]), 0.5 * (bounds[0] + bounds[2]))
            
            # Create map
            new_map = folium.Map(location=center, tiles=self._tiles, zoom_start=6)
            parent_map = new_map
            
        else:
            
            parent_map = add_to_map
                       
        # Add layer
        tooltip = folium.GeoJsonTooltip(fields=tooltip_attributes)

        if varname is None:
            style_function = ColormapStyleFunction(cmap, varname, randomcolors=True)
        else:
            style_function = ColormapStyleFunction(cmap, varname)

        folium.GeoJson(self._json_dataset,
                       style_function=style_function,
                       tooltip=tooltip,
                       name="Test").add_to(parent_map)

        if varname is not None:
            
            # Add colorbar
            colormap = cmap.to_step(n=4)
            colormap.caption = varname
            colormap.add_to(parent_map)

        #if add_to_map is None:
            #parent_map.fit_bounds(self._dataset.total_bounds.tolist())
        
        if add_to_map is None:
            return new_map
        
rmap._dataset['wse_swot']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbars[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['all']]=Qbars[str(reachid)]
    #if not algo[str(reachid)]:
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['algo']]=algo[str(reachid)]
    #if not np.isnan(
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['gauge_id']]=ohio_gauges[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['slope_sword']]=slopes[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['slope_swot']]=slope_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['dark_frac_swot']]=dark_frac_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['width_swot']]=width_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['wse_swot']]=wse_swot_oh[str(reachid)]

rmap._dataset['wse'] = np.round(rmap._dataset['wse'],2)
rmap._json_dataset = rmap._dataset.to_json()
all_map = rmap.get_centerlines_map(varname="wse_swot",varlimits=[80,800],cmap=["red","orange", "yellow", "green", "blue", "purple"]
                                   ,tooltip_attributes=['reach_id','all','algo','wse','wse_swot', 'width', 'width_swot','river_name', 'gauge_id', 
                                                        'slope_sword', 'slope_swot', 'dark_frac_swot']) 
all_map

In [ ]:
pd.DataFrame(rmap._dataset)['slope_swot'].max()

In [ ]:
from matplotlib import cm
import numpy as np
import random

class ColormapStyleFunction:
    """Object to handle colormap style functions
    """
    
    def __init__(self, cmap, attribute,randomcolors=False):
        self._cmap = cmap
        self._attribute = attribute
        self._randomcolors = randomcolors
        
    def __call__(self, x):
        if self._randomcolors:
            #hexcolor = '#ff0000'
            hexcolor="#"+''.join([random.choice('0123456789ABCDEF') for i in range(6) ] )
        else:
            hexcolor = self._cmap(x["properties"][self._attribute])

        return {'color': hexcolor, 'weight' : 3}
    
def get_centerlines_map(self, varname=None, cmap=None, tooltip_attributes=None, add_to_map=None, varlimits=[None, None]):
        """Build a map width reaches as centerlines colored with values of a variable
        
        Parameters
        ----------
        varname : str
            Name of the variable used for coloring
        width_attribute : str
            Name of the variable for the width
        cmap : branca.Colormap
            Colormap used for coloring
        tooltip_attributes : list or None
            List of variables to display using ToolTip
        """
        
        # Set default values for unset parameters
        if cmap is None and varname is not None:
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            #cmap = branca.colormap.linear.YlOrRd_09.scale(self._dataset[varname].min(),
            #                                              self._dataset[varname].max())
            cmap = branca.colormap.linear.YlOrRd_09.scale(varlimits[0],
                                                          varlimits[1])
            #cmap = branca.colormap.ColorMap(vmin=0.0, vmax=1.0, caption='', max_labels=10)
            #cmap = branca.colormap.LinearColormap(['blue', 'yellow', 'red']).scale(varlimits[0], varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])
            
        elif isinstance(cmap, list):
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            cmap = branca.colormap.LinearColormap(cmap).scale(varlimits[0],
                                                              varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])

        if tooltip_attributes is None:
            if varname is None:
                tooltip_attributes = ["reach_id"]
            else:
                tooltip_attributes = ["reach_id", varname]

        if add_to_map is None:
        
            # Retrieve bounding box and center
            bounds = self._dataset.geometry.total_bounds.tolist()
            center = (0.5 * (bounds[1] + bounds[3]), 0.5 * (bounds[0] + bounds[2]))
            
            # Create map
            new_map = folium.Map(location=center, tiles=self._tiles, zoom_start=6)
            parent_map = new_map
            
        else:
            
            parent_map = add_to_map
                       
        # Add layer
        tooltip = folium.GeoJsonTooltip(fields=tooltip_attributes)

        if varname is None:
            style_function = ColormapStyleFunction(cmap, varname, randomcolors=True)
        else:
            style_function = ColormapStyleFunction(cmap, varname)

        folium.GeoJson(self._json_dataset,
                       style_function=style_function,
                       tooltip=tooltip,
                       name="Test").add_to(parent_map)

        if varname is not None:
            
            # Add colorbar
            colormap = cmap.to_step(n=4)
            colormap.caption = varname
            colormap.add_to(parent_map)

        #if add_to_map is None:
            #parent_map.fit_bounds(self._dataset.total_bounds.tolist())
        
        if add_to_map is None:
            return new_map
        
rmap._dataset['dark_frac_swot']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbars[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['all']]=Qbars[str(reachid)]
    #if not algo[str(reachid)]:
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['algo']]=algo[str(reachid)]
    #if not np.isnan(
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['gauge_id']]=ohio_gauges[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['slope_sword']]=slopes[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['slope_swot']]=slope_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['dark_frac_swot']]=dark_frac_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['width_swot']]=width_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['wse_swot']]=wse_swot_oh[str(reachid)]

rmap._dataset['wse'] = np.round(rmap._dataset['wse'],2)
rmap._json_dataset = rmap._dataset.to_json()
all_map = rmap.get_centerlines_map(varname="dark_frac_swot",varlimits=[0,1],cmap=["red","orange", "yellow", "green", "blue"]
                                   ,tooltip_attributes=['reach_id','all','algo','wse','wse_swot', 'width', 'width_swot','river_name', 'gauge_id', 
                                                        'slope_sword', 'slope_swot', 'dark_frac_swot']) 
all_map

In [ ]:
from matplotlib import cm
import numpy as np
import random

class ColormapStyleFunction:
    """Object to handle colormap style functions
    """
    
    def __init__(self, cmap, attribute,randomcolors=False):
        self._cmap = cmap
        self._attribute = attribute
        self._randomcolors = randomcolors
        
    def __call__(self, x):
        if self._randomcolors:
            #hexcolor = '#ff0000'
            hexcolor="#"+''.join([random.choice('0123456789ABCDEF') for i in range(6) ] )
        else:
            hexcolor = self._cmap(x["properties"][self._attribute])

        return {'color': hexcolor, 'weight' : 3}
    
def get_centerlines_map(self, varname=None, cmap=None, tooltip_attributes=None, add_to_map=None, varlimits=[None, None]):
        """Build a map width reaches as centerlines colored with values of a variable
        
        Parameters
        ----------
        varname : str
            Name of the variable used for coloring
        width_attribute : str
            Name of the variable for the width
        cmap : branca.Colormap
            Colormap used for coloring
        tooltip_attributes : list or None
            List of variables to display using ToolTip
        """
        
        # Set default values for unset parameters
        if cmap is None and varname is not None:
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            #cmap = branca.colormap.linear.YlOrRd_09.scale(self._dataset[varname].min(),
            #                                              self._dataset[varname].max())
            cmap = branca.colormap.linear.YlOrRd_09.scale(varlimits[0],
                                                          varlimits[1])
            #cmap = branca.colormap.ColorMap(vmin=0.0, vmax=1.0, caption='', max_labels=10)
            #cmap = branca.colormap.LinearColormap(['blue', 'yellow', 'red']).scale(varlimits[0], varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])
            
        elif isinstance(cmap, list):
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            cmap = branca.colormap.LinearColormap(cmap).scale(varlimits[0],
                                                              varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])

        if tooltip_attributes is None:
            if varname is None:
                tooltip_attributes = ["reach_id"]
            else:
                tooltip_attributes = ["reach_id", varname]

        if add_to_map is None:
        
            # Retrieve bounding box and center
            bounds = self._dataset.geometry.total_bounds.tolist()
            center = (0.5 * (bounds[1] + bounds[3]), 0.5 * (bounds[0] + bounds[2]))
            
            # Create map
            new_map = folium.Map(location=center, tiles=self._tiles, zoom_start=6)
            parent_map = new_map
            
        else:
            
            parent_map = add_to_map
                       
        # Add layer
        tooltip = folium.GeoJsonTooltip(fields=tooltip_attributes)

        if varname is None:
            style_function = ColormapStyleFunction(cmap, varname, randomcolors=True)
        else:
            style_function = ColormapStyleFunction(cmap, varname)

        folium.GeoJson(self._json_dataset,
                       style_function=style_function,
                       tooltip=tooltip,
                       name="Test").add_to(parent_map)

        if varname is not None:
            
            # Add colorbar
            colormap = cmap.to_step(n=4)
            colormap.caption = varname
            colormap.add_to(parent_map)

        #if add_to_map is None:
            #parent_map.fit_bounds(self._dataset.total_bounds.tolist())
        
        if add_to_map is None:
            return new_map
        
rmap._dataset['slope_swot']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbars[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['all']]=Qbars[str(reachid)]
    #if not algo[str(reachid)]:
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['algo']]=algo[str(reachid)]
    #if not np.isnan(
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['gauge_id']]=ohio_gauges[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['slope_sword']]=slopes[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['slope_swot']]=slope_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['dark_frac_swot']]=dark_frac_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['width_swot']]=width_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['wse_swot']]=wse_swot_oh[str(reachid)]

rmap._dataset['wse'] = np.round(rmap._dataset['wse'],2)
rmap._json_dataset = rmap._dataset.to_json()
all_map = rmap.get_centerlines_map(varname="slope_swot",varlimits=[-0.01,0.015],cmap=["red","orange", "yellow", "green", "blue", "purple", "pink"]
                                   ,tooltip_attributes=['reach_id','all','algo','wse','wse_swot', 'width', 'width_swot','river_name', 'gauge_id', 
                                                        'slope_sword', 'slope_swot', 'dark_frac_swot']) 
all_map

In [ ]:
from matplotlib import cm
import numpy as np
import random

class ColormapStyleFunction:
    """Object to handle colormap style functions
    """
    
    def __init__(self, cmap, attribute,randomcolors=False):
        self._cmap = cmap
        self._attribute = attribute
        self._randomcolors = randomcolors
        
    def __call__(self, x):
        if self._randomcolors:
            #hexcolor = '#ff0000'
            hexcolor="#"+''.join([random.choice('0123456789ABCDEF') for i in range(6) ] )
        else:
            hexcolor = self._cmap(x["properties"][self._attribute])

        return {'color': hexcolor, 'weight' : 3}
    
def get_centerlines_map(self, varname=None, cmap=None, tooltip_attributes=None, add_to_map=None, varlimits=[None, None]):
        """Build a map width reaches as centerlines colored with values of a variable
        
        Parameters
        ----------
        varname : str
            Name of the variable used for coloring
        width_attribute : str
            Name of the variable for the width
        cmap : branca.Colormap
            Colormap used for coloring
        tooltip_attributes : list or None
            List of variables to display using ToolTip
        """
        
        # Set default values for unset parameters
        if cmap is None and varname is not None:
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            #cmap = branca.colormap.linear.YlOrRd_09.scale(self._dataset[varname].min(),
            #                                              self._dataset[varname].max())
            cmap = branca.colormap.linear.YlOrRd_09.scale(varlimits[0],
                                                          varlimits[1])
            #cmap = branca.colormap.ColorMap(vmin=0.0, vmax=1.0, caption='', max_labels=10)
            #cmap = branca.colormap.LinearColormap(['blue', 'yellow', 'red']).scale(varlimits[0], varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])
            
        elif isinstance(cmap, list):
            if varlimits[0] is None:
                varlimits[0]= self._dataset[varname].min()
            if varlimits[1] is None:
                varlimits[1]= self._dataset[varname].max()

            cmap = branca.colormap.LinearColormap(cmap).scale(varlimits[0],
                                                              varlimits[1])
            #cmap = branca.colormap.StepColormap(["r", "y", "g", "c", "b", "m"])

        if tooltip_attributes is None:
            if varname is None:
                tooltip_attributes = ["reach_id"]
            else:
                tooltip_attributes = ["reach_id", varname]

        if add_to_map is None:
        
            # Retrieve bounding box and center
            bounds = self._dataset.geometry.total_bounds.tolist()
            center = (0.5 * (bounds[1] + bounds[3]), 0.5 * (bounds[0] + bounds[2]))
            
            # Create map
            new_map = folium.Map(location=center, tiles=self._tiles, zoom_start=6)
            parent_map = new_map
            
        else:
            
            parent_map = add_to_map
                       
        # Add layer
        tooltip = folium.GeoJsonTooltip(fields=tooltip_attributes)

        if varname is None:
            style_function = ColormapStyleFunction(cmap, varname, randomcolors=True)
        else:
            style_function = ColormapStyleFunction(cmap, varname)

        folium.GeoJson(self._json_dataset,
                       style_function=style_function,
                       tooltip=tooltip,
                       name="Test").add_to(parent_map)

        if varname is not None:
            
            # Add colorbar
            colormap = cmap.to_step(n=4)
            colormap.caption = varname
            colormap.add_to(parent_map)

        #if add_to_map is None:
            #parent_map.fit_bounds(self._dataset.total_bounds.tolist())
        
        if add_to_map is None:
            return new_map
        
rmap._dataset['width_swot']=-1.

for reachid in reachids_oh:
    if not np.isnan(Qbars[str(reachid)]):
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['all']]=Qbars[str(reachid)]
    #if not algo[str(reachid)]:
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['algo']]=algo[str(reachid)]
    #if not np.isnan(
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['gauge_id']]=ohio_gauges[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['slope_sword']]=slopes[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['slope_swot']]=slope_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['dark_frac_swot']]=dark_frac_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['width_swot']]=width_swot_oh[str(reachid)]
        rmap._dataset.loc[rmap._dataset['reach_id'].astype(str)==str(reachid),['wse_swot']]=wse_swot_oh[str(reachid)]

rmap._dataset['wse'] = np.round(rmap._dataset['wse'],2)
rmap._json_dataset = rmap._dataset.to_json()
all_map = rmap.get_centerlines_map(varname="width_swot",varlimits=[-100,2000],cmap=["red","orange", "yellow", "green", "blue"]
                                   ,tooltip_attributes=['reach_id','all','algo','wse','wse_swot', 'width', 'width_swot','river_name', 'gauge_id', 
                                                        'slope_sword', 'slope_swot', 'dark_frac_swot']) 
all_map

## Close output file

In [ ]:
results.close()
priors.close()